In [ ]:
pip install ipython-sql

La connexion a aucune base de données n'est effectuée. La création des bases de données est effectuée de façon temporaire au sein de ce Notebook.

In [ ]:
%load_ext sql
%sql sqlite://

Les tables sont crées via des "magic methods" permettant d'utiliser SQL au sein de cellule du Notebook.

In [3]:
%%sql 
CREATE TABLE Users
    (id INTEGER,
    email TEXT,
    password TEXT,
    birthdate DATE);
    
CREATE TABLE Categories
    (id INTEGER,
    label TEXT);
    
CREATE TABLE Topics
    (id INTEGER,
    title TEXT);
    
CREATE TABLE Posts
    (id INTEGER,
    postDate DATE,
    content TEXT);

 * sqlite://
Done.
Done.
Done.
Done.


[]

`date` du module `datetime` sera utilisé pour déterminer la date des posts.

In [ ]:
from datetime import date

La première classe utilisée est `User`. Celle-ci contient un disctionnaire `USERS` qui contient l'ensemble des utilisateurs. La clé de chaque instance d'utilisateur correspond à son identifiant.  

La classe contient (pour le moment) 3 méthodes de classe:  
`User.create_user` permet de créer un nouvel utilisateur. Le mot de passe n'est pas encodé (bien qu'il pourait l'être).  
`User.delete_user` permet de supprimer un utilisateur en saisissant son identifiant.  
`User.add_post` permet de créer un nouveau post, et met à jour la table correspondante.
La classe `User` contient un attribut `topics` qui contient l'ensemble des instance de `Topic` pour chaque instance de `User` ainsi qu'un attribut `posts` qui contient l'ensemble des instance de `Post` pour chaque instance de `User`.

In [ ]:
class User:
    
    USERS = {}
    
    def __init__(self, id, email, password, birthdate):
        self.id = id
        self.email = email
        self.password = password
        self.birthdate = birthdate
        self.topics = []
        self.posts = []
        
    @classmethod  
    def create_user(cls, email, password, birthdate):
        # Generate a new ID according to the number of users:
        id = len(cls.USERS) + 1
        # The ID is updated if it already exists:
        while  id in cls.USERS.keys(): 
            id += 1
        new_user = User(id, email, password, birthdate)
        cls.USERS[id] = new_user
        
        %sql INSERT INTO Users (id, email, password, birthdate) VALUES (:id, :email, :password, :birthdate);
        
    @classmethod
    def delete_user(cls, id):
        del cls.USERS[id]
        
        %sql DELETE FROM Users WHERE id = :id;
            
    @classmethod       
    def add_post(cls, user_id, content, topic):
        post_date = str(date.today())
        id = len(Post.POSTS) + 1
        while  id in Post.POSTS.keys():
            id += 1
        new_post = Post(id, post_date, content, topic, user_id)
        Post.POSTS[id] = new_post
        cls.USERS[user_id].posts.append(new_post)
        if topic not in cls.USERS[user_id].topics:
            cls.USERS[user_id].topics.append(topic)
        if topic not in Topic.TOPICS.values():
            Topic._create_topic(user_id, topic)
            
        %sql INSERT INTO Posts (id, postDate, content) VALUES (:id, :post_date, :content);

3 autres classes ont été créés : `Category`, `Topic`, et `Post`.  

La classe `Category` contient un attribut `topics` qui contient l'ensemble des instance de `Topic` pour chaque instance de `Category`.

La classe `Topic` contient un attribut `posts` qui contient l'ensemble des instance de `Post` pour chaque instance de `Topic`.

In [7]:
 class Category:
    
    CATEGORIES = {}
    
    def __init__(self, id, label):
        self.id = id
        self.label = label
        self.topics = []
        
    @classmethod  
    def create_category(cls, label):
        # Generate a new ID according to the number of users:
        id = len(cls.CATEGORIES) + 1
        # The ID is updated if it already exists:
        while  id in cls.CATEGORIES.keys(): 
            id += 1
        new_category = Category(id, label)
        cls.CATEGORIES[id] = new_category
        
        %sql INSERT INTO Categories (id, label) VALUES (:id, :label);
        
    @classmethod
    def delete_category(cls, id):
        del cls.CATEGORIES[id]
        
        %sql DELETE FROM Categories WHERE id = :id;
         
        
class Topic:
    TOPICS = {}
    def __init__(self, id, title, user, category = None):
        self.id = id
        self.title = title
        self.category = category
        self.user = user
        self.posts = []
    
    @classmethod
    def _create_topic(cls, user_id, title):
        id = len(cls.TOPICS) + 1
        # The ID is updated if it already exists:
        while  id in cls.TOPICS.keys(): 
            id += 1
        new_topic = Topic(id, title, user_id)
        cls.TOPICS[id] = new_topic
        
        %sql INSERT INTO Topics (id, title) VALUES (:id, :title);
        
        
          
class Post: 
    POSTS = {}
    def __init__(self, id, post_date, content, topic, user):
        self.id = id
        self.post_date = post_date
        self.content = content
        self.topic = topic
        self.user = user
    